# Análisis de Clasificación Internacional de Enfermedades

## Integrantes - Grupo 15

- Sara Sofia Cárdenas Rodríguez - 202214907
- Daniel Felipe Diaz Moreno - 202210773
- Juan Sebastián Urrea López - 201914710


## Los datos

Se trabaja con un conjunto de datos que contiene los codigos de enfemerdades de la Clasificación Internacional de Enfermedades (CIE) con sus correspondientes significados


## 1. Carga y Manipulación básica de los Datos

In [43]:
# Importaciones de bibliotecas
import pandas as pd
!pip  install xlrd


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [44]:
cie_df = pd.read_excel("Cie.xls", skiprows=6)


print(f"Las dimensiones del archivo son: {cie_df.shape}")
cie_df.head(10)

Las dimensiones del archivo son: (12423, 4)


,COD_3,DESRIPCION CATEGORIAS DE TRES CARACTERES,COD_4,DESCRIPCION CODIGOS DE CUATRO CARACTERES
0,A00,COLERA,A000,"COLERA DEBIDO A VIBRIO CHOLERAE O1, BIOTIPO CH..."
1,NaN,NaN,A001,"COLERA DEBIDO A VIBRIO CHOLERAE O1, BIOTIPO EL..."
2,NaN,NaN,A009,COLERA NO ESPECIFICADO
3,A01,FIEBRES TIFOIDEA Y PARATIFOIDEA,A010,FIEBRE TIFOIDEA
4,NaN,NaN,A011,FIEBRE PARATIFOIDEA A
5,NaN,NaN,A012,FIEBRE PARATIFOIDEA B
6,NaN,NaN,A013,FIEBRE PARATIFOIDEA C
7,NaN,NaN,A014,"FIEBRE PARATIFOIDEA, NO ESPECIFICADA"
8,A02,OTRAS INFECCIONES DEBIDAS SALMONELLA,A020,ENTERITIS DEBIDA A SALMONELLA
9,NaN,NaN,A021,SEPTICEMIA DEBIDA A SALMONELLA


Los unicos atributos de interés son COD_4 y DESCRIPCION CODIGOS DE CUATRO CARACTERES, asi que se eliminarán los demás atributos

In [45]:
print(cie_df.columns)
columnasCie = ['COD_4', 'DESCRIPCION CODIGOS DE CUATRO CARACTERES']
cie_df[columnasCie].head(10)

Index(['COD_3', 'DESRIPCION CATEGORIAS DE TRES CARACTERES', 'COD_4',
       'DESCRIPCION CODIGOS DE CUATRO CARACTERES'],
      dtype='object')


,COD_4,DESCRIPCION CODIGOS DE CUATRO CARACTERES
0,A000,"COLERA DEBIDO A VIBRIO CHOLERAE O1, BIOTIPO CH..."
1,A001,"COLERA DEBIDO A VIBRIO CHOLERAE O1, BIOTIPO EL..."
2,A009,COLERA NO ESPECIFICADO
3,A010,FIEBRE TIFOIDEA
4,A011,FIEBRE PARATIFOIDEA A
5,A012,FIEBRE PARATIFOIDEA B
6,A013,FIEBRE PARATIFOIDEA C
7,A014,"FIEBRE PARATIFOIDEA, NO ESPECIFICADA"
8,A020,ENTERITIS DEBIDA A SALMONELLA
9,A021,SEPTICEMIA DEBIDA A SALMONELLA


In [46]:
cie_df[columnasCie].describe()

,COD_4,DESCRIPCION CODIGOS DE CUATRO CARACTERES
count,12423,12423
unique,12423,12420
top,Z999,SIFILIS CARDIOVASCULAR
freq,1,2


Se puede ver que los codigos son unicos (debido a que el dato que más se repite tiene frecuencia de 1). Sin embargo, parece ser que las descripciones no son unicas.

## 2. Entendimiento de los datos

## 2.1. Análisis de calidad de datos





## 2.1.1. Análisis de completitud
A continuación, se presentan los porcentajes de completitud e incompletitud para cada uno de los atributos.

In [47]:
porcentajeIncompletitudAtributo =  (100 * cie_df[columnasCie].isnull().sum() / cie_df[columnasCie].shape[0]).sort_values(ascending=False)
pd.DataFrame({
    "Atributo": porcentajeIncompletitudAtributo.index,
    "Completitud (%)": 100 - porcentajeIncompletitudAtributo.values,
    "Incompletitud (%)": porcentajeIncompletitudAtributo.values
})

,Atributo,Completitud (%),Incompletitud (%)
0,COD_4,100.0,0.0
1,DESCRIPCION CODIGOS DE CUATRO CARACTERES,100.0,0.0


Se puede ver que todos los codigos tienen su correspondiente descripcion y no existen codigos vacios, no se requiere hacer arreglos por completitud.

## 2.1.2. Análisis de unicidad
A continuación, se buscará determinar cuántas filas duplicadas hay en el dataset

In [48]:
registrosDuplicados = cie_df[columnasCie].duplicated()
totalDuplicados = registrosDuplicados.sum()
totalOpiniones = cie_df[columnasCie].shape[0]
porcentajeDuplicados = 100.0 * totalDuplicados / totalOpiniones
print(f"Se cuenta con {totalDuplicados} registros duplicados ({porcentajeDuplicados:.2f}%)")



Se cuenta con 0 registros duplicados (0.00%)


Como se habia evidenciado, tampoco hay registros duplicados, no se requiere hacer arreglos por unicidad.

## 2.1.3. Análisis de validez

In [49]:
descripciones_duplicadas = cie_df.groupby('DESCRIPCION CODIGOS DE CUATRO CARACTERES').filter(lambda x: len(x) > 1)
descripciones_duplicadas

,COD_3,DESRIPCION CATEGORIAS DE TRES CARACTERES,COD_4,DESCRIPCION CODIGOS DE CUATRO CARACTERES
238,A52,SIFILIS TARDIA,A520†,SIFILIS CARDIOVASCULAR
3527,I98*,OTROS TRASTORNOS DEL SISTEMA CIRCULATORIO EN E...,I980*,SIFILIS CARDIOVASCULAR
6438,NaN,NaN,Q318,OTRAS MALFORMACIONES CONGENITAS DE LA LARINGE
6482,NaN,NaN,Q388,OTRAS MALFORMACIONES CONGENITAS DE LA LARINGE
6900,R09,OTROS SINTOMAS Y SIGNOS QUE INVOLUCRAN LOS SIS...,R090,ASFIXIA
8311,T71,ASFIXIA,T71,ASFIXIA


Se puede ver que las dos primeras filas, si bien tienen la misma descripción, son de enfermedades distintas, por tanto deberian tratarse como tal. Lo mismo pasa con las dos ultimos filas.

Al revisar manualmente, se evidencia que Q388 hace referencia a "OTRAS MALFORMACIONES CONGENITAS DE LA LENGUA, DE LA BOCA Y DE LA FARINGE" mientras que Q318 hace referencia a "MALFORMACIONES CONGENITAS DE LA LARINGE". Por tanto deben considerarse como enfermedades diferentes y no habrian errores de validez

## 2.1.4. Análisis de consistencia

No se encontraron problemas de consistencia en los datos

# 4. Cambios propuestos

# Dim_Muerte

**Fuente de datos**
Cie

Campos
- COD_4	
- DESCRIPCION CODIGOS DE CUATRO CARACTERES

**Extraer informacion de**  
Cie

**Transformaciones**

1. Renombrar COD_4 a cod_causa_basica
2. Renombrar DESCRIPCION CODIGOS DE CUATRO CARACTERES a causa_basica


**Carga de datos**
Destino de datos: Dim_Muerte

Campos:
- Cod_causa_basica  
- Causa_basica   
